In [ ]:
import pandas as pd

DIRECTORY = "challenge_dataset/"

X_train = pd.read_csv(DIRECTORY + "X_train.csv", sep=';').drop(columns=['Id'])
X_train

,Caption
0,de mourir avant l'heure\n de ne plus revoir me...
1,la maladie pour les autres et pour moi\n et le...
2,Comment vont s'en sortir ceux qui sont mal ou ...
3,"Inquiétude pour la santé de mes proches, pour ..."
4,"Bien entendu contracter la maladie,"
...,...
480,avoir une forme grave du civid
481,Inquiétude de ne pas retrouver une liberté d'a...
482,L'incertitude dans laquelle nous sommes. \n Ne...
483,"inquiétude normale face à une épidémie, craint..."


In [ ]:
y_train = pd.read_csv(DIRECTORY + "y_train.csv", sep=';').drop(columns=['Id'])
y_train

,category_1,category_2,category_3,category_4
0,1,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0
...,...,...,...,...
480,1,0,0,0
481,0,0,0,1
482,0,1,0,1
483,0,1,0,0


In [ ]:
import nltk

In [ ]:
def return_token(sentence):
    # Tokeniser la phrase
    doc = nltk.word_tokenize(sentence)
    # Retourner le texte de chaque token
    return [X.text for X in doc]

In [ ]:
for line in X_train['Caption']:
    print(return_token(line))

NameError: name 'X_train' is not defined